# Section 5 — GroupBy, Aggregations, and Transformations

### Goal
Understand how to split, apply, and combine data using Pandas’ powerful `groupby()` mechanism — enabling aggregation, transformation, and advanced analytics.

By the end of this section, you will:
- Master the **split–apply–combine** pattern.
- Compute descriptive summaries and custom aggregates.
- Apply multiple aggregations simultaneously.
- Use `transform()` and `apply()` for feature creation.
- Solve real analytical problems using grouped data.

## Subtopics
1. The Split–Apply–Combine paradigm.
2. Simple aggregation with `groupby()`.
3. Multi-column grouping and custom aggregation functions.
4. Transformations vs Aggregations.
5. Real-world applications and performance tips.

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(42)

# Simulate a sales dataset
data = {
    'Region': np.random.choice(['North', 'South', 'East', 'West'], 20),
    'SalesRep': np.random.choice(['Alice', 'Bob', 'Charlie', 'Diana', 'Ethan'], 20),
    'Month': np.random.choice(['Jan', 'Feb', 'Mar', 'Apr'], 20),
    'UnitsSold': np.random.randint(10, 100, 20),
    'UnitPrice': np.random.randint(50, 150, 20)
}
sales = pd.DataFrame(data)
sales['Revenue'] = sales['UnitsSold'] * sales['UnitPrice']
sales.head()

## 1️⃣ The Split–Apply–Combine Paradigm

This concept underlies `groupby()`:
- **Split:** divide data into groups (by key columns)
- **Apply:** perform a computation on each group
- **Combine:** merge results into a single DataFrame

In [ ]:
# Example: average revenue per region
sales.groupby('Region')['Revenue'].mean()

## 2️⃣ Simple Aggregations
Pandas supports built-in aggregations like `sum()`, `mean()`, `count()`, `min()`, `max()`, and more.

In [ ]:
# Basic stats by region
agg_basic = sales.groupby('Region')['Revenue'].agg(['mean', 'sum', 'count'])
agg_basic

## 3️⃣ Multi-Column Grouping and Custom Aggregations
You can group by multiple keys and perform multiple aggregations per column.

In [ ]:
# Group by Region and Month
agg_multi = sales.groupby(['Region', 'Month']).agg({
    'UnitsSold': ['sum', 'mean'],
    'Revenue': ['sum', 'max']
})
agg_multi

In [ ]:
# Rename multi-level columns for clarity
agg_multi.columns = ['_'.join(col).strip() for col in agg_multi.columns.values]
agg_multi.reset_index(inplace=True)
agg_multi.head()

## 4️⃣ Custom Aggregation Functions
You can define your own aggregation logic using lambdas or named functions.

In [ ]:
# Custom aggregation — revenue variance per region
custom_agg = sales.groupby('Region')['Revenue'].agg(
    Total='sum',
    Average='mean',
    RevenueRange=lambda x: x.max() - x.min()
)
custom_agg

## 5️⃣ Transformations vs Aggregations

- **Aggregation:** reduces each group to a single value.
- **Transformation:** returns a new series of the same size as the original, enabling per-row operations.

In [ ]:
# Normalize revenue within each region (z-score like transformation)
sales['Revenue_z'] = sales.groupby('Region')['Revenue'].transform(
    lambda x: (x - x.mean()) / x.std()
)
sales.head()

In [ ]:
# Calculate each rep's performance as % of their region's average
sales['PerformancePct'] = (
    sales['Revenue'] / sales.groupby('Region')['Revenue'].transform('mean') * 100
)
sales.head()

## ✅ Real-World Problem 1 — Regional Sales Summary Dashboard

**Scenario:** You’re preparing a dashboard summarizing sales performance.
Compute per-region:
- Total and average revenue
- Average units sold
- Count of active sales reps
- Month with the highest total sales

In [ ]:
regional_summary = sales.groupby('Region').agg(
    TotalRevenue=('Revenue', 'sum'),
    AvgRevenue=('Revenue', 'mean'),
    AvgUnits=('UnitsSold', 'mean'),
    SalesReps=('SalesRep', pd.Series.nunique)
)

# Find month with highest regional sales
top_month = sales.groupby(['Region', 'Month'])['Revenue'].sum().reset_index()
top_month = top_month.loc[top_month.groupby('Region')['Revenue'].idxmax()]
top_month.rename(columns={'Month': 'TopMonth', 'Revenue': 'TopMonthRevenue'}, inplace=True)

dashboard = pd.merge(regional_summary, top_month[['Region', 'TopMonth', 'TopMonthRevenue']], on='Region')
dashboard

## ✅ Real-World Problem 2 — Detecting High-Variance Reps

**Goal:** Identify sales representatives whose revenue consistency deviates sharply within their region.

**Steps:**
1. Compute revenue standard deviation for each SalesRep.
2. Compare to region’s average std.
3. Flag reps exceeding 1.5× the regional std as ‘Volatile’.

In [ ]:
rep_stats = sales.groupby(['Region', 'SalesRep'])['Revenue'].agg(['mean', 'std']).reset_index()
region_std = rep_stats.groupby('Region')['std'].transform('mean')
rep_stats['VolatilityFlag'] = np.where(rep_stats['std'] > 1.5 * region_std, 'Volatile', 'Stable')
rep_stats

## ⚙️ Under the Hood: How GroupBy Works
- `groupby()` creates a `GroupBy` object — an iterator over grouped subsets.
- It stores a mapping of group labels → row indices (hash tables internally).
- Aggregations are dispatched to optimized Cython code for speed.
- Transformations maintain index alignment — returning Series aligned to the original DataFrame.
- Multi-aggregations build MultiIndex columns, which can be flattened for readability.

## 💡 Best Practices / Common Pitfalls
- Avoid calling `.groupby()` repeatedly in loops — group once and reuse.
- Always reset index (`.reset_index()`) for clean DataFrames.
- For readability, use **named aggregations** (`agg(new_name=('col', 'func'))`).
- Watch for NaNs — aggregations skip them silently.
- Multi-level columns from `.agg()` can be flattened with list comprehensions.

## 🧩 Challenge Exercise

**Dataset:** You have a DataFrame `transactions` with columns:
`['CustomerID', 'ProductCategory', 'Region', 'Amount', 'Discount', 'Date']`.

**Tasks:**
1. Compute per-region, per-category total sales and average discount.
2. Find each customer’s average spend compared to their region’s average (as %).
3. Tag regions as **“High Revenue”** if total sales > median of all regions.
4. Create a summary DataFrame sorted by total revenue descending.

_Hint:_ Use `groupby().transform()` for normalization and `rank()` for ordering.

# --- End of Section 5 — Continue to Section 6 ---